In [2]:
# !pip install deplacy stanza
import stanza
#----------------------
#!pip install deplacy trankit transformers
import trankit
#----------------------
#!pip install deplacy spacy-udpipe
import spacy_udpipe
#----------------------
#!pip install deplacy spacy_jptdp
import spacy_jptdp
#----------------------
#!pip install --index-url https://pypi.clarin-pl.eu/simple deplacy combo
#!pip install combo
import combo
#import combo.predict
#----------------------
#!pip install deplacy camphr en-udify@https://github.com/PKSHATechnology-Research/camphr_models/releases/download/0.7.0/en_udify-0.7.tar.gz
import pkg_resources,imp
imp.reload(pkg_resources)
import spacy

What is the purpose of syntactic analysis?
Its purpose is to understand the structure of input text, from the smallest basic symbols, all the way to sentences, and then derive logical meaning from it.

Syntactic analysis is an extremely important aspect of natural language processing (NLP) because it assists in figuring out the grammatical meaning of any sentence. 

In [3]:
# Import file
import codecs
# Stop words
#!pip install advertools
import advertools as adv
# Presenting a semantic analysis
import deplacy
import re
import string
import pandas as pd
# Text samerization
#!pip install summa
import summa
from summa import summarizer
from summa import keywords

2022-07-15 02:23:41,467 | INFO | textcleaner.py:12 | <module> | 'pattern' package not found; tag filters are not available for English


In [6]:
class HebrewDataProcessing():
    def load_data(self, file_name):
        # Load dataSet
        data = list(codecs.open(file_name, 'r', 'utf-8').readlines())
        df =pd.DataFrame({'text':data})
        return df
    
    def get_syntactic_analysis(self, type, df):
        # Select a library type to preform data syntax analysis
        if type=='stanza':
            #stanza.download('he') 
            nlp=stanza.Pipeline("he")
        elif type=='trankit':
            nlp=trankit.Pipeline("hebrew")
        elif type=='spacy-udpipe':
            #spacy_udpipe.download("he")
            nlp=spacy_udpipe.load("he")
        elif type=='spacy-jptdp':
            nlp=spacy_jptdp.load("he_htb")
        elif type == 'combo-pytorch':
            nlp=combo.predict.COMBO.from_pretrained("hebrew-ud27")
        elif type=='camphr-udify':
            nlp=spacy.load("en_udify")
        #doc=nlp(str(df))
        #deplacy.render(doc,WordRight=True)
        #deplacy.serve(doc,port=None,RtoL=True)
        return nlp
    
    def get_tokens(self, doc):
        # list of upos
        upos =["CCONJ","PUNCT", "ADP", "PRON", "DET", "SCONJ", "NUM", "ADV"]
        # Remove specific upos
        list_of_tokens = [str(word.lemma) for sent in doc.sentences for word in sent.words  if word.upos not in upos]
        return [list_of_tokens] 


    def data_clearing(self, df):
        # remove urls
        df["text"] = df["text"].apply(lambda x: re.split('https:\/\/.*', str(x))[0]) 
        # remove numbers and eng words 
        df["text"] = df["text"].apply(lambda x:re.sub('[0-9a-zA-Z+]',' ', str(x)))
        # Removing html tags and amoji
        df["text"] = df["text"].apply(lambda x:re.sub(r'<[^>]*>', '', str(x))) 
        amoji ='[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)]'
        df["text"] = df["text"].apply(lambda i:re.sub(amoji+'+','', str(i))) 
        #---------------------------------------
        # tokenize
        nlp = self.get_syntactic_analysis('stanza', df["text"])
        df_nlp = df["text"].apply(nlp)
        df_nlp = df_nlp.apply(lambda r: pd.Series(self.get_tokens(r), index=['tokens']))
        #---------------------------------------
        # list of stopwords by the spaCy package
        word_tokens = list(adv.stopwords['hebrew']) 
        # remove stopwords
        df_nlp['tokens']=df_nlp['tokens'].apply(lambda x: [word_token for word_token in x if word_token not in word_tokens])
        #---------------------------------------
        # convert list to string
        df_nlp['tokens'] = (df_nlp['tokens'].transform(lambda x: " ".join(map(str,x))))        
        return df_nlp['tokens']
        
if __name__ == "__main__":
    obj = HebrewDataProcessing()
    #Read dataset
    df= obj.load_data('data/hebrew_text.tsv')
    df=df.head(100)
    df_nlp = obj.data_clearing(df)
    df_nlp.to_csv('data/preprocessing_hebrew.csv', index = False, header=False)
    df=df.head(100)
    print(df_nlp.head())
    def hebrow_summarizer(str_text):
        print('Text summary:\n', summarizer.summarize(str_text))
        print('keywords:\n', keywords.keywords(str_text))
        
    str_text = df_nlp.to_string()
    hebrow_summarizer(str_text)


2022-07-15 02:28:08 INFO: Loading these models for language: he (Hebrew):
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |

2022-07-15 02:28:08,569 | INFO | core.py:112 | __init__ | Loading these models for language: he (Hebrew):
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |

2022-07-15 02:28:08 INFO: Use device: cpu
2022-07-15 02:28:08,572 | INFO | core.py:123 | __init__ | Use device: cpu
2022-07-15 02:28:08 INFO: Loading: tokenize
2022-07-15 02:28:08,573 | INFO | core.py:129 | __init__ | Loading: tokenize
2022-07-15 02:28:08 INFO: Loading: mwt
2022-07-15 02:28:08,584 | INFO | core.py:129 | __init__ | Loading: mwt
2022-07-15 02:28:08 INFO: Loading: pos
2022-07-15 02:28:08,605 | INFO | core.py:129 | __init__ | Loading: pos
2022-07-15 02:28:08 INFO: 

0                     כאב בכה משפחה תדע צער
1                                   איש יקר
2                            כבוד מון הצלחה
3    תל חי רובי עצב קרן אור תקוה נשיא ישראל
4                       נקי כף בר לבב הצלחה
Name: tokens, dtype: object
Text summary:
 3                תל חי רובי עצב קרן אור תקוה נשיא ישראל
7     כבוד נשיא התאים עולם מלהי נשיא מדינה ישראל ישר...
8                 כבוד אדון נשיא איש ימין חשב אהבה ניצח
17                     ווא ריגש עזרה ׳ נשיא מדינה ישראל
27    כבוד נשיא בא טוב שמח פגישה עתידי אבו מאזן בקש ...
29                              כבוד נשיא כבוד תמך מילה
30                                        כוח כבוד נשיא
45                       חיוך כבש מדינה ישראל כבוד נשיא
46                      נשיא מדינה ישראל כלבבי יישר כוח
47                 מילה כדורבן גא רובי נשיא מדינה ישראל
66                        יח נשיא חדש תפארת מדינה ישראל
72                בריאות כבוד נשיא הלך לנכד משפחה ישראל
74    פוסט ראה ראוי תואר כבוד נשיא מדינה ראובן רובי ...
79      כ